In [1]:
import csv
import math
import os
import pandas as pd
import cv2
import numpy as np
from tkinter import *

In [2]:
# Check made for correct file extemsion 
def checkExtension(extension, fileName):
    root, ext = os.path.splitext(fileName)
    if ext != extension:
        # Not passed a file with the correct extension
        print('Not a suitable .csv file. The file recieved was: %s' % fileName)
        return False
    #File has correct extension
    return True

In [3]:
# Preform the calculations
def calculateMetrics(threshold, csvFile):
    # import csv file using pandas
    df = pd.read_csv(csvFile, skiprows= 1)
    r, c = df.shape
    magnitude_list = []
    time_list = []

    # Traverse the data frame and preform calculations on the data. 
    for ind in range(1, r-1):
        # Checks if likelihood is higher than the set threshold
        if float(df.at[ind + 1, 'Vole1_RightEye.2']) < threshold or float(df.at[ind, 'Vole1_RightEye.2']) < threshold:
            # Point was not above likeliness threshold, leave blank 
            magnitude_list.append(None)
        else:
            # Both points are above the likeliness threshold 
            # Vector distance Vole 1
            magnitude_vector_vole1 = math.sqrt(math.pow(float(df.at[ind + 1, 'Vole1_RightEye']) - float(df.at[ind, 'Vole1_RightEye']),2) 
                                    + math.pow(float(df.at[ind + 1, 'Vole1_RightEye.1']) - float(df.at[ind, 'Vole1_RightEye.1']),2))
            # Vector distance Vole 2
            magnitude_vector_vole2 = math.sqrt(math.pow(float(df.at[ind + 1, 'Vole2_RightEye']) - float(df.at[ind, 'Vole2_RightEye']),2) 
                                    + math.pow(float(df.at[ind + 1, 'Vole2_RightEye.1']) - float(df.at[ind, 'Vole2_RightEye.1']),2))

            # Change in vector magnitude
            magnitude_vector = magnitude_vector_vole1 - magnitude_vector_vole2

            # Append to calculations list
            magnitude_list.append(magnitude_vector)

        # Append timestamps
        time_list.append(ind)

    return magnitude_list

In [4]:
def confirmBehavior():
    # New window
    popup = Tk() 
    # Set passed title and change to all caps
    popup.wm_title("Behavior Check")
    # Set default window geometry to 500 x 100
    popup.geometry("400x200") 
    # Set text for label to passed message
    label = Label(popup, text="Was mating behavior displayed?", font=("Open Sans", 12))
    # Place on window
    label.pack(side="top", fill="x", pady=10)
    # Create button to close window
    B1 = Button(popup, text="Yes")
    # Pack on window too
    B1.pack()
    # Create button to close window
    B2 = Button(popup, text="No")
    # Pack on window too
    B2.pack()
    # Wait the code until popup window is destroyed
    mainloop()

In [5]:
def windowedPreview(threshold, movementList, video):
    for i, item in enumerate(movementList):
        if item == None:
            pass
        elif item <= threshold and item >= -threshold:
            video.set(1, i-1)
            #print("%s\n" % item)
            # Capture frame-by-frame
            ret, frame = video.read()
            if ret == True:
                # Display the resulting frame
                cv2.imshow('Frame',frame)

                # Press Q on keyboard to  exit
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            # Break the loop
            else: 
                break

In [6]:
# Main handler function 
def analyzeCsvFile(threshold, csvFile):
    # check file extension
    if not checkExtension('.csv', csvFile):
        return None
    else:
        # Prints if was given a .csv file 
        print('Pulling data from %s' % csvFile)
    
    # Return calculations made in the function call
    return calculateMetrics(threshold, csvFile)


In [7]:
def main():
    movement_threshold = 0.5
    covariation_threshold = 20

    # Holds the user inputed file paths
    behavior_csv = r'C:\Users\Behavior Scoring\Desktop\DLC Project Utils\csv files\similar_motionDeepCut_resnet50_Normal_MotionAug14shuffle1_110000.csv'

    # Holds the correlated video to the csv file
    behavior_video = r'C:\Users\Behavior Scoring\Desktop\DLC Project Utils\labled videos\DLC_Mating_Vod.mp4'
    
    # Create a VideoCapture object and read from input file
    # If the input is the camera, pass 0 instead of the video file name
    cap = cv2.VideoCapture(behavior_video)

    # Check if camera opened successfully
    if (cap.isOpened()== False): 
      print("Error opening video stream or file")
    
    # Pass files as arguments to the function; Return data structure with analysis metrics
    analyzed_csv_data = analyzeCsvFile(movement_threshold, behavior_csv)

    # Show glimpses for windows under threshold
    windowedPreview(covariation_threshold, analyzed_csv_data, cap)
    
    # Added a human aspect to the video
    print(confirmBehavior())
    
    # When everything done, release the video capture object
    cap.release()
 
    # Closes all the frames
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Pulling data from C:\Users\Behavior Scoring\Desktop\DLC Project Utils\csv files\similar_motionDeepCut_resnet50_Normal_MotionAug14shuffle1_110000.csv
None
